In [79]:
import numpy as np
from scipy.stats import poisson, norm, bernoulli, gamma
from scipy.special import psi
nneuron = 41
sprefs = np.linspace(-60, 60, nneuron)

sig0_sq      = float(3**2)
sig1_sq      = float(12**2)
sigtc_sq     = float(10**2)
ndatapergain = 3000
mu = 0

tau_0 = 1/sig0_sq
tau_1 = 1/sig1_sq
d_tau = tau_0 - tau_1
sig_tc = 10
min_angle = -60
max_angle = 60
step = 3
prefact = np.sqrt(tau_0/tau_1)

def generate_popcode_data(ndatapergain, nneuron, sig0_sq, sig1_sq, sigtc_sq, gain, noise):
    s_s0 = np.random.normal(mu, np.sqrt(sig0_sq), ndatapergain/2)
    s_s1 = np.random.normal(mu, np.sqrt(sig1_sq), ndatapergain/2)
    s = np.concatenate((s_s0, s_s1), axis=1)
    sprefs_data = np.tile(sprefs, (ndatapergain, 1))
    r = gain * np.exp(-np.square((np.transpose(np.tile(s, (nneuron, 1))) - sprefs_data))/(2 * sigtc_sq))
    if noise == "poisson":
        r = np.random.poisson(r) + 0.0
    ar = np.sum(r, axis=1)/sigtc_sq
    br = np.sum(r*sprefs_data, axis=1)/sigtc_sq
    
    term_1 = (1 + sig1_sq * ar) / (1 + sig0_sq  * ar)
    term_2_num = (sig1_sq - sig0_sq) * br**2
    term_2_denom = 2 * (1 + sig0_sq  * ar) * (1 + sig1_sq * ar)
    xpnt_a = .5 * np.log(term_1) - term_2_num/term_2_denom
    p = 1/(1 + np.exp(-xpnt_a))
    return r, p, ar, br

In [187]:
r, p, ar, br = generate_popcode_data(ndatapergain, nneuron, sig0_sq, sig1_sq, sigtc_sq, 1, "poisson")
def logistic(eta):
    return 1/(1 + np.exp(-eta))
def norm_g(zTw):
    return 0, zTw + 3
def relu(zTw):
    return max(0, zTw)

k_l = [nneuron, 1, 1]
L = len(k_l)
xi_gamma1 = np.random.rand(nneuron, 1) #[mu, sigma]
xi_normal = np.random.rand(1, 2) #[mu, sigma]
xi_gamma2 = np.random.rand(1, 1) #[shape]
xi = [xi_gamma1, xi_normal, xi_gamma2]
q_W_dist = [np.random.normal, np.random.normal, np.random.gamma]
q_z_dist = [np.random.poisson, np.random.normal, np.random.binomial]
p_W_lik = [gamma, norm, gamma]
p_z_lik = [poisson, norm, bernoulli]
nonlin = [logit, relu, norm_g]
train_data = [r[1]]
prior = .5
rho = .2
print train_data

S = 1
z = [None]*L
W = [None]*L
p_z = [None]*L
q_z = [None]*L
p_W = [None]*L
q_W = [None]*L

def sample_z(q_z_dist, lam, S):
    z = []
    for params in lam:
        if q_z_dist == np.random.poisson:
            lam_z = params
            z.append(q_z_dist(lam=lam_z, size=S))
        elif q_z_dist == np.random.normal:
            loc_z, scale_z = params
            z.append((q_z_dist(loc=loc_z, scale=scale_z, size=S)))
        elif q_z_dist == np.random.binomial:
            p_z = params
            z.append(q_z_dist(n=1, p=p_z, size=S))
    return np.concatenate(z)

def sample_W(q_W_dist, xi, S):
    W = []
    for params in xi:
        if q_W_dist == np.random.normal:
            loc_W, scale_W = params
            W.append(q_W_dist(loc=loc_W, scale=scale_W, size=S))
        if q_W_dist == np.random.gamma:
            shape_W = params
            W.append(q_W_dist(shape=shape_W, size=S))
    return np.concatenate(W)

def g_poisson(z, lam):
    return -1 + z/float(lam)

def g_norm(z, loc, scale):
    d_loc = (z - loc)/scale
    d_scale = -(1/(2 * scale)) + ((z - loc)**2)/(2 * scale**2)
    return d_loc, d_scale

def g_bernoulli(z, eta):
    return 2 * z - 1 * np.exp(-(2 * z - 1) * eta)/(1 + np.exp(-(2 * z - 1) * eta))

def g_gamma(z, shape):
    return -psi(shape) + np.log(z)
                
for n in train_data:
    converged = False
    lam_poisson = np.random.rand(nneuron, 1) #[ln lmbda]
    lam_normal = np.random.rand(1, 2) #[mu, sigma]
    lam_bernoulli = np.random.rand(1, 1) #[p]
    lam = [lam_poisson, lam_normal, lam_bernoulli]
    while not converged:
        for l in range(L):
            z[l] = sample_z(q_z_dist[l], lam[l], S)
            W[l] = sample_W(q_W_dist[l], xi[l], S)
            p_z[l] = [None] * len(z[l])
            q_z[l] = [None] * len(z[l])
            
            #Unnecessary?
            """
            p_W[l] = [None] * len(W[l])
            q_W[l] = [None] * len(W[l])
            """
            
            params = z[l] * W[l]
            for i in range(len(z[l])):
                if p_z_lik[l] == poisson:
                    lam_p = nonlin[l](params[i])
                    if l == 0:
                        p_z[l][i] = p_z_lik[l].pmf(n[i], lam_p)
                    else:
                        p_z[l][i] = p_z_lik[l].pmf(z[l-1][i], lam_p)
                elif p_z_lik[l] == norm:
                    loc_p, scale_p = nonlin[l](params[i])
                    if l == 0:
                        p_z[l][i] = p_z_lik[l].pdf(x=n[i], loc=loc_p, scale=scale_p)
                    else:
                        p_z[l][i] = p_z_lik[l].pdf(x=z[l-1][i], loc=loc_p, scale=scale_p)
                elif p_z_lik[l] == bernoulli:
                    p_p = nonlin[l](params[i])
                    if l == 0:
                        p_z[l][i] = p_z_lik[l].pmf(n[i], p_p)
                    else:
                        p_z[l][i] = p_z_lik[l].pmf(z[l-1][i], p_p)
                if q_z_dist[l] == np.random.poisson:
                    lam_q = nonlin[l](lam[l][i])
                    q_z[l][i] = poisson.pmf(z[l][i], lam_q)
                    g[l][i] = g_poisson(z[l][i], lam_q)
                elif q_z_dist[l] == np.random.normal:
                    loc_q, scale_q = nonlin[l](lam[l][i])
                    q_z[l][i] = norm.pdf(x=z[l][i], loc=loc_q, scale=scale_q)
                    g[l][i] = g_norm(z[l][i], loc_q, scale_q)
                elif q_z_dist[l] == np.random.binomial:
                    p_q = nonlin[l](lam[l][i])
                    q_z[l][i] = bernoulli.pmf(z[l][i], p_q)
                    g[l][i] = g_bernoulli(z[l][i], lam[l][i])
                
                #Weights
                #p_w = q_w if same  distribution?
                """
                if p_W_lik[l] == norm:
                    loc_p, scale_p = xi[l][i]
                    p_W[l][i] = p_W_lik[l].pdf(x=W[l][i], loc=loc_p, scale=scale_p)
                if p_W_lik[l] == gamma:
                    shape_p = xi[l][i]
                    p_W[l][i] = p_W_lik[l].pdf(x=W[l][i], shape=shape_p)
                """
                if q_W_dist[l] == np.random.normal:
                    loc_q, scale_q = xi[l][i]
                    #q_W[l][i] = norm.pdf(x=W[l][i], loc=loc_q, scale=scale_q)
                    g_W[l][i] = g_norm(W[l][i], loc_q, scale_q)
                if q_W_dist[l] == np.random.gamma:
                    shape_q = xi[l][i]
                    #q_W[l][i] = gamma.pdf(x=W[l][i], shape=shape_p)
                    g_W[l][i] = g_gamma(W[l][i], shape_q)
            print p_z
            p_z[l] = np.log(p_z[l])
            q_z[l] = np.log(q_z[l])
        p_z[L] = p
        l = 0
        for l in range(L):
            S = [None] * len(z[l])
            T = [None] * len(z[l])
            for k in range(len(z[l])):
                S[k] = g[l][k] * np.add(np.add(p[l], p[l+1][k]), q[l][k]) #???
                lam[l][k] += rho * S
            T[k] = g_W[l][k] * p[l] #???
            xi[l] += rho * T
        converged = True
    print "z"
    print z_p
    print z_q
    print W_p
    print W_q

[array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  2.,  0.,  2.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.])]
[[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.70470556885418767, 0.5, 0.5, 0.5, 0.5, 0.5, 0.84133586367158208, 0.5, 0.5, 0.5, 0.12011325347955037, 0.82151119382295679, 0.024617983416998074, 0.3465735902799727, 0.5, 0.5, 0.77917245994642148, 0.28878239699568969, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.41971635799701046, 0.5, 0.5], None, None]
[array([-0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69314718,
       -0.69314718, -0.3499752 , -0.69314718, -0.69314718, -0.69314718,
       -0.69314718, -0.69314718, -0.17276434, -0.69314718, -0.69314718,
       -0.69314718, -2.1193202 , -0.19660972, -3.70427807, -1.0596601 ,
       -0.69314718, -0.69314718, -0.24952287, -1.24208183, -0.69314718,
       -0.69314718, -0.69314718, -0.693

/Users/ocnc2015/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:107: RuntimeWarning: divide by zero encountered in log
/Users/ocnc2015/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:108: RuntimeWarning: divide by zero encountered in log


In [108]:
s = "Id"
s == "Id"

True

In [192]:
np.add([1, 0, 2, 1, 3], 3)

array([4, 3, 5, 4, 6])